# Othello Probcut

### Initiale Konfiguration

Importieren von Abhängigkeiten und Konfiguration

In [20]:
%run othello_lib.ipynb

In [12]:
import math
import copy

from tkinter import TclError

# STRATEGIES
HUMAN = 'H'
TRIVIAL = 'T'
MINIMAX = 'M'
ALPHABETA = 'A'
PROBCUT = 'P'

# PLAYERS
BLACK_PLAYER = ALPHABETA
WHITE_PLAYER = ALPHABETA

### Hilfsfunktionen
Diese Hilfsfunktionen werden von mehreren KI's verwendet

Die Funktion <tt>terminal_utility(state)</tt> gibt für einen Spielzustand einen Wert zurück der repräsentiert, ob und welcher Spieler gewonnen hat. Hat kein Spieler gewonnen, wird der Wert 0 zurückgegeben, ansonsten 1 oder -1 für den maximierenden oder minimierenden Spieler.

In [13]:
def terminal_utility(state):
    winner = state.return_winner()
    if(winner is None):
        return 0
    else:
        return 1 if winner == state.turn else -1

Die Funktion <tt>heuristic_utility(state)</tt> berechnet eine Heuristik für einen Spielzustand zwischen -1 und 1.

In [14]:
def heuristic_utility(state):
    player_cells = state.get_total_cells(state.turn)
    opponent_cells = state.get_total_cells(state._opposite_turn(state.turn))
    return (player_cells - opponent_cells) / 64

### Triviale KI
Diese KI iteriert reihenweise über das Spielfeld und spielt den ersten gültigen Zug

In [15]:
def trivial_ai_make_move(state):
    for row in range(state.rows):
        for col in range(state.cols):
            try:
                state.move(row, col)
                return
            except InvalidMoveException:
                pass


### Minimax KI
Diese KI verwendet den Minimax Algorithmus

In [16]:
MINIMAX_DEPTH_LIMIT = 4

def minimax(state, depth, player):
    global minimax_best_move
    if(state.is_game_over()):
        return terminal_utility(state) * player
    if(depth >= MINIMAX_DEPTH_LIMIT):
        return heuristic_utility(state) * player
    utility = -math.inf
    for move in state.get_possible_moves():
        tmp_state = copy.deepcopy(state)
        tmp_state.move(move[0], move[1])
        tmp_utility = -minimax(tmp_state, depth + 1, -player)
        if(tmp_utility > utility):
            utility = tmp_utility
            if(depth == 0):
                minimax_best_move = move
    return utility

def minimax_ai_make_move(state):
    minimax(state, 0, 1)
    state.move(minimax_best_move[0], minimax_best_move[1])

### Alpha-Beta KI
Diese KI verwended den Minimax Algorithmus mit Alpha-Beta Pruning

In [27]:
ALPHABETA_DEPTH_LIMIT = 6

def alphabeta(state, depth, alpha, beta, player):
    global alphabeta_best_move
    if(state.is_game_over()):
        return terminal_utility(state) * player
    if(depth >= ALPHABETA_DEPTH_LIMIT):
        return heuristic_utility(state) * player
    utility = -math.inf #* player
    for move in state.get_possible_moves():
        tmp_state = copy.deepcopy(state)
        tmp_state.move(move[0], move[1])
        tmp_utility = -alphabeta(tmp_state, -beta, -alpha, depth + 1, -player)
        if(tmp_utility > utility):
            utility = tmp_utility
            if(depth == 0):
                alphabeta_best_move = move
        if(utility >= beta):
            return utility
        alpha = max(alpha, utility)
    return max_utility

def alphabeta_ai_make_move(state):
    alphabeta(state, -math.inf, math.inf, 0, 1)
    state.move(alphabeta_best_move[0], alphabeta_best_move[1])

### ProbCut KI
An dieser Stelle beginnt die Implementierung der Künstlichen Intelligenz mittels des Minimax Algorithmus und ProbCut

In [28]:
def probcut_ai_make_move(state):
    for row in range(state.rows):
        for col in range(state.cols):
            try:
                state.move(row, col)
                return
            except InvalidMoveException:
                pass

### Applikation Starten

Führen sie zum Started der Applikations den folgenden Code aus.

In [29]:
print('Application start') 
gui = OthelloGUI()

try:
    while 1:
        gui._root_window.update()
        game_state = gui._game_state
        if (not game_state.is_game_over()):
            # Check if/which AI is playing
            strat = BLACK_PLAYER if game_state.turn == BLACK else WHITE_PLAYER
            if (strat != HUMAN): 
                # Execute AI strategy for current player
                if(strat == TRIVIAL):
                    trivial_ai_make_move(game_state)
                elif(strat == MINIMAX):
                    minimax_ai_make_move(game_state)
                elif(strat == ALPHABETA):
                    alphabeta_ai_make_move(game_state)
                elif(strat == PROBCUT):
                    probcut_ai_make_move(game_state)
                else:
                    raise Exception('Invalid Strategy')
                    
                # Update GUI after AI move
                gui._board.update_game_state(game_state)
                gui._board.redraw_board()
                gui._black_score.update_score(game_state)
                gui._white_score.update_score(game_state)

                if game_state.is_game_over():
                    gui._player_turn.display_winner(game_state.return_winner())
                else:
                    gui._player_turn.switch_turn(game_state)
except TclError:
    print('Application exit')
except Exception as e:
    gui._root_window.destroy()
    raise e

Application start
Application exit


### Testing code

In [ ]:
test_board = OthelloGameState()

In [ ]:
minimax_ai_make_move(test_board)
test_board.current_board

In [ ]:
import cProfile

cProfile.run('minimax_ai_make_move(test_board)')
test_board.current_board

# Legacy code

In [ ]:
def alphabeta_max(state, alpha, beta, depth):
    global alphabeta_best_move
    if(state.is_game_over()):
        return terminal_utility(state)
    if(depth >= ALPHABETA_DEPTH_LIMIT):
        return heuristic_utility(state)
    max_utility = -math.inf
    for move in state.get_possible_moves():
        tmp_state = copy.deepcopy(state)
        tmp_state.move(move[0], move[1])
        tmp_utility = alphabeta_min(tmp_state, alpha, beta, depth + 1)
        if(tmp_utility > max_utility):
            max_utility = tmp_utility
            if(depth == 0):
                alphabeta_best_move = move
        if(max_utility >= beta):
            return max_utility
        alpha = max(alpha, max_utility)
    return max_utility

def alphabeta_min(state, alpha, beta, depth):
    global alphabeta_best_move
    if(state.is_game_over()):
        return -terminal_utility(state)
    if(depth >= ALPHABETA_DEPTH_LIMIT):
        return -heuristic_utility(state)
    min_utility = math.inf
    for move in state.get_possible_moves():
        tmp_state = copy.deepcopy(state)
        tmp_state.move(move[0], move[1])
        tmp_utility = alphabeta_max(tmp_state, alpha, beta, depth + 1)
        if(tmp_utility < min_utility):
            min_utility = tmp_utility
            if(depth == 0):
                alphabeta_best_move = move
        if(min_utility <= alpha):
            return min_utility
        beta = min(beta, min_utility)
    return min_utility